In [1]:
!pip install transformers
!pip install torchsummary
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import ElectraModel, ElectraConfig, ElectraForMaskedLM, ElectraTokenizer, PretrainedConfig, ElectraForTokenClassification, BertTokenizerFast, DataCollatorForWholeWordMask, DataCollatorForLanguageModeling, BertForMaskedLM, AutoTokenizer
# from torchsummary import summary
from datasets import load_dataset
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

# Some tests, you can directly go to next part

In [3]:
config = ElectraConfig()
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')


discriminator = ElectraModel.from_pretrained('google/electra-small-discriminator')
inputs = tokenizer("hello my dog is cute", return_tensors="pt")
#print(inputs)
outputs = discriminator(**inputs, )
last_hidden_states = outputs.last_hidden_state
#print(last_hidden_states, last_hidden_states.shape)

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")
print(inputs, labels)
model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator')
outputs = model(**inputs, labels=labels["input_ids"])
loss = outputs.loss
logits = outputs.logits
print(torch.argmax(logits, dim = -1))
print(tokenizer.decode(torch.squeeze(torch.argmax(logits, dim = -1)).tolist()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])} {'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003, 3000, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

tensor([[1012, 1996, 3007, 1997, 2605, 2003, 3000, 1012, 1012]])
. the capital of france is paris..


In [4]:
print(tokenizer.mask_token_id)

103


In [5]:
print(discriminator)

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0-11): 12 x ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((

In [6]:
"""generator_config = {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": True,
  "transformers_version": "4.36.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}

discriminator_config = {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": True,
  "transformers_version": "4.36.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}"""

'generator_config = {\n  "attention_probs_dropout_prob": 0.1,\n  "embedding_size": 128,\n  "hidden_act": "gelu",\n  "hidden_dropout_prob": 0.1,\n  "hidden_size": 256,\n  "initializer_range": 0.02,\n  "intermediate_size": 1024,\n  "layer_norm_eps": 1e-12,\n  "max_position_embeddings": 512,\n  "model_type": "electra",\n  "num_attention_heads": 4,\n  "num_hidden_layers": 12,\n  "pad_token_id": 0,\n  "position_embedding_type": "absolute",\n  "summary_activation": "gelu",\n  "summary_last_dropout": 0.1,\n  "summary_type": "first",\n  "summary_use_proj": True,\n  "transformers_version": "4.36.2",\n  "type_vocab_size": 2,\n  "use_cache": True,\n  "vocab_size": 30522\n}\n\ndiscriminator_config = {\n  "attention_probs_dropout_prob": 0.1,\n  "embedding_size": 128,\n  "hidden_act": "gelu",\n  "hidden_dropout_prob": 0.1,\n  "hidden_size": 256,\n  "initializer_range": 0.02,\n  "intermediate_size": 1024,\n  "layer_norm_eps": 1e-12,\n  "max_position_embeddings": 512,\n  "model_type": "electra",\n  "n

# Load dataset

In [12]:
#Input = masked data
#label = real data

#input = prediction of generator
#output = binary output

#We will masked 20% of the input

#103 is the encoding for [MASK]

# def masking(x , mask_token = 103):
#     k = int(len(x)*1/10)
#     ids = random.sample(range(1, len(x)-1), k)
#     for i in ids:
#         x[i] = mask_token
#     return x, ids


model_ckpt = "papluca/xlm-roberta-base-language-detection"
# model_ckpt = "bert-base-multilingual-cased"
# model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# tokenizer = BertTokenizerFast.from_pretrained(model_ckpt)
dataset = load_dataset("papluca/language-identification")
# dataset = load_dataset("imdb")

# def preprocessing_fn(x, tokenizer):
#     generator_labels = tokenizer(
#         x["text"],
#         add_special_tokens=False,
#         truncation=False,
#         padding=False,
#         return_attention_mask=False,
#     )
#     # x["input_ids"], masked_ids = masking(generator_labels.copy(), 103)
#     x["input_ids"] = generator_labels['input_ids']
#     x['token_type_ids'] = generator_labels['token_type_ids']
#     # x["labels"] = generator_labels['input_ids']

#     # attention_mask = torch.ones((len(x["input_ids"])))
#     # x["attention_mask"] = attention_mask
#     # x["masked_ids"] = masked_ids
#     return x

def preprocess_function(examples):
    #  return = tokenizer(
    #     x["text"],
    #     add_special_tokens=False,
    #     truncation=False,
    #     padding=False,
    #     return_attention_mask=False,
    # )
    return tokenizer(examples['text'], add_special_tokens=True)

print(len(dataset['train']))

n_samples = 100  # the number of training example

# We first shuffle the data !
dataset = dataset.shuffle()

# Select n_samples
train_dataset = dataset['train'].select(range(n_samples))
valid_dataset = dataset['validation'].select(range(n_samples//5))

# Tokenize the dataset
train_dataset = train_dataset.map(
    preprocess_function, remove_columns=dataset["train"].column_names, batched=True, 
)
valid_dataset = valid_dataset.map(
    preprocess_function, remove_columns=dataset["train"].column_names, batched=True,
)


# Remove useless columns
# train_dataset = train_dataset.select_columns(["input_ids", "attention_mask", "labels", "masked_ids"])
# valid_dataset = valid_dataset.select_columns(["input_ids", "attention_mask", "labels", "masked_ids"])

block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result
train_dataset = train_dataset.map(group_texts, batched=True)
valid_dataset = valid_dataset.map(group_texts, batched=True)
# train_dataset = train_dataset.select_columns(["input_ids", "token_type_ids"])
# valid_dataset = valid_dataset.select_columns(["input_ids", "token_type_ids"])


70000


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [13]:
index = 0
print(train_dataset['input_ids'][index])
tokenizer.decode(train_dataset['input_ids'][index])
# print(len(train_dataset['input_ids'][0]))
# print(train_dataset['masked_ids'][0])
# print(train_dataset['labels'][0])

[0, 242, 1129, 242, 60384, 242, 60384, 242, 705, 242, 45824, 923, 42425, 6, 5, 2, 0, 384, 12976, 39, 101100, 1000, 889, 18818, 127, 54403, 2251, 38135, 117357, 25403, 160966, 550, 16180, 7991, 7669, 26986, 6325, 128788, 78360, 224, 132406, 681, 889, 2261, 6924, 2494, 16151, 550, 2550, 4069, 37294, 550, 3616, 308, 6, 4, 2550, 61260, 67820, 9985, 3822, 6800, 550, 11472, 3042, 6, 5, 2, 0, 1720, 28332, 5543, 21, 42093, 5977, 198, 51, 280, 5708, 1158, 5, 42313, 960, 137255, 5, 6, 225103, 46356, 88, 10975, 5, 2, 0, 53779, 3526, 8, 3725, 280, 246, 11, 15, 1702, 71147, 1388, 152, 6, 4, 92293, 1707, 120061, 2262, 13035, 11411, 6, 4, 236270, 29, 88658, 40229, 6, 4, 49, 163, 23525, 6, 4, 27902, 239820]


"<s>'ju'nini'nini '?'kaka alitoka.</s><s> Tạm dừng trong một khoảnh khắc tại sân thượng phía tây của cung điện bên cạnh phòng trưng bày des hẹn cho một quan điểm đầu tiên của sự điều hòa của anh ta, sự sắp xếp bất hợp pháp của đất nước.</s><s> Para llegar hasta la función radio es un peñazo. Está mal diseñado. Necesita mejorar el software.</s><s> феста de san pedoa ( 29 юни ) :, християнски поклон пред петър, покровител на рибарите, в melaka, където португал"

In [73]:
# class DataCollator:
#     def __init__(self, tokenizer):
#         self.tokenizer = tokenizer

#     def __call__(self, batch):
#         max_label_length = max(len(feature["input_ids"]) for feature in batch)
#         max_length_ids = max(len(feature["masked_ids"]) for feature in batch)
#         for feature in batch:
#             current_length = len(feature["input_ids"])
#             current_length_mask = len(feature["masked_ids"])
#             remainder = [tokenizer.pad_token_id] * (max_label_length - current_length)

#             feature["input_ids"] = feature["input_ids"] + remainder
#             feature["labels"] = feature["input_ids"][1:]
#             feature["labels"] = [
#                 feature["labels"][i] if i < current_length - 1 else -100
#                 for i in range(max_label_length - 1)
#             ]
#             feature["input_ids"] = feature["input_ids"][:-1]
#             feature["attention_mask"] = [
#                 1 if x < current_length else 0 for x in range(max_label_length - 1)
#             ]
#             feature["masked_ids"] = [
#                 feature["masked_ids"][i] if i < current_length_mask - 1 else -1
#                 for i in range(max_length_ids - 1)
#             ]
#         features = {
#             "input_ids": torch.tensor([f["input_ids"] for f in batch]),
#             "attention_mask": torch.tensor([f["attention_mask"] for f in batch]),
#             "labels": torch.tensor([f["labels"] for f in batch]),
#             "masked_ids": torch.tensor([f["masked_ids"] for f in batch]),
#         }
#         return features

# print(train_dataset[0].keys())
# print(len(train_dataset[0]["input_ids"]), len(train_dataset[0]["attention_mask"]), len(train_dataset[0]["labels"]))
# print(len(train_dataset[1]["input_ids"]), len(train_dataset[1]["attention_mask"]), len(train_dataset[1]["labels"]))
# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=.15)

batch_size = 32

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, collate_fn=data_collator, pin_memory=True, pin_memory_device='cuda:0'
)
valid_dataloader = DataLoader(
    valid_dataset, batch_size=batch_size, collate_fn=data_collator, pin_memory=True, pin_memory_device='cuda:0'
)
n_valid = len(valid_dataset)
n_train = len(train_dataset)

batch = next(iter(train_dataloader))
print(batch["input_ids"].shape, batch["labels"].shape, batch["attention_mask"].shape)

NotImplementedError: Could not run 'aten::_pin_memory' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_pin_memory' is only available for these backends: [MPS, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDi

In [15]:
print(batch)

{'input_ids': tensor([[     0,    242,   1129,  ...,      4,  27902, 239820],
        [  5231, 151884,   1698,  ...,  42687,      2,      0],
        [  4239,  35702,      6,  ...,   4377, 250001, 250001],
        ...,
        [    56,  90365,   5885,  ..., 250001,  91857, 250001],
        [  3352, 250001,   4742,  ...,   6573,   9568,  62106],
        [ 56629, 250001, 121678,  ..., 149497,  21636, 126120]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100, 20322,  1920],
        ...,
        [ -100,  -100,  -100,  ..., 43774,  -100, 15861],
        [ -100, 34941,  -100,  ...,  -100,  -100,  -100],
        [ -100, 24853,  -100,  ...,  

In [16]:
print(batch['input_ids'][2])
print(batch['labels'][2])

tensor([  4239,  35702,      6, 144901, 250001, 193753,      6,   4377,      6,
        250001,    287,   5450,    996,      6, 250001,      6,  69650,   1302,
        250001,  10345,   1472,      6,   4377, 250001, 250001,    629,   8663,
          4050,    287, 250001,  62091, 250001, 250001,    460,      6,      4,
          4993,      6, 159104,    871, 193753,      6,   4377,      6,  15525,
           287,   5450,    996,      6,   4377,      6,  69650,   1302,    968,
         36048,   5450,    996,      6,   4377,      6,  28416,      6,   4377,
          1618,   4050,  12829,  36048, 104081,    471,  17092,    629,   8663,
          4050, 250001,   1812, 250001, 250001, 250001,    460,      6,      4,
         51344, 250001,    996,      6,   4377,      6, 250001,   1302,    471,
         10093,  72618,    646,  35870, 250001,      6,  27885,      6,    976,
        131409,    471,  17092,    629,   8663,   5199, 250001,  23097,      6,
          4377,   3418,  11646,   3418, 

# Model instantiation

In [83]:
class GeneratorHead(nn.Module):
    """Prediction module for the generator, made up of two dense layers."""
    def __init__(self, config):
        super().__init__()

        self.LayerNorm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        self.dense = nn.Linear(config.hidden_size, config.embedding_size)
        self.dense2 = nn.Linear(config.embedding_size, config.vocab_size)

    def forward(self, generator_hidden_states):
        hidden_states = self.dense(generator_hidden_states)
        hidden_states = torch.nn.GELU()(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        hidden_states = self.dense2(hidden_states)
        hidden_states = torch.nn.Softmax(dim = -1)(hidden_states)

        return hidden_states

class Generator(nn.Module):
    """Complete generator module"""
    def __init__(self, generator_body, generator_head):
        super().__init__()
        self.generator_body = generator_body
        self.generator_head = generator_head

    def forward(self, input, attention_masks):
      output = self.generator_body(input, attention_masks)
      output = self.generator_head(output.last_hidden_state)
      return output

class DiscriminatorHead(nn.Module):
    """Discriminator module for the generator, made up of two dense layers."""
    def __init__(self, config):
        super().__init__()

        self.LayerNorm = nn.LayerNorm(config.embedding_size, eps=config.layer_norm_eps)
        self.dense = nn.Linear(config.hidden_size, config.embedding_size)
        self.dense2 = nn.Linear(config.embedding_size, 2)

    def forward(self, discriminator_hidden_states):
        hidden_states = self.dense(discriminator_hidden_states)
        hidden_states = torch.nn.GELU()(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        hidden_states = self.dense2(hidden_states)

        return hidden_states

class Discriminator(nn.Module):
    """Complete Discriminator"""
    def __init__(self, discriminator_body, discriminator_head):
        super().__init__()
        self.discriminator_body = discriminator_body
        self.discriminator_head = discriminator_head

    def forward(self, input):
      output = self.discriminator_body(input).last_hidden_state
      output = self.discriminator_head(output)
      return output

class ELECTRALoss():
  def __init__(self, loss_weights=(1.0, 50.0)):
    self.loss_weights = loss_weights
    self.gen_loss_fc = nn.CrossEntropyLoss(ignore_index = -100)
    self.disc_loss_fc = nn.BCEWithLogitsLoss()

  def __call__(self, generator_pred, discriminator_pred, generator_labels, masked_ids):
    list_gen_pred = []
    list_gen_labels = []
    for i in range(masked_ids.shape[0]):
      for j in range(masked_ids.shape[1]):
        masked_index = masked_ids[i,j]
        list_gen_pred.append(torch.unsqueeze(generator_pred[i,masked_index,:],dim =0))
        list_gen_labels.append(torch.unsqueeze(generator_labels[i,masked_index], dim = 0))
    list_gen_pred = torch.concat(list_gen_pred, axis = 0)
    list_gen_labels = torch.concat(list_gen_labels, axis = 0)
    gen_loss = self.gen_loss_fc(list_gen_pred, list_gen_labels)
    gen_acc = torch.sum(torch.argmax(list_gen_pred, dim = -1) == list_gen_labels)/ len(list_gen_labels)*100

    generated_tokens = torch.argmax(generator_pred, dim = -1)

    discriminator_labels =torch.ones((discriminator_pred.shape[0], discriminator_pred.shape[1])).cuda()
    for i in range(masked_ids.shape[0]):
      for j in range(masked_ids.shape[1]):
        masked_index = masked_ids[i,j]
        if generated_tokens[i,masked_index] != generator_labels[i,masked_index]:
          discriminator_labels[i,masked_index] = 0
    discriminator_labels =  discriminator_labels.to(torch.int64).view(discriminator_labels.shape[0]*discriminator_labels.shape[1]).float()
    discriminator_pred = discriminator_pred.view(discriminator_pred.shape[0]*discriminator_pred.shape[1],discriminator_pred.shape[2])
    disc_loss = self.disc_loss_fc(discriminator_pred, discriminator_labels)
    a = torch.argmax(discriminator_pred, dim = -1)
    print(a.shape, discriminator_labels.shape)
    disc_acc = torch.sum(torch.argmax(discriminator_pred, dim = -1) == discriminator_labels)/ len(discriminator_labels)*100
    return gen_loss * self.loss_weights[0] + disc_loss * self.loss_weights[1], gen_acc, disc_acc

In [85]:
generator_config = ElectraConfig(vocab_size = tokenizer.vocab_size, embedding_size= 64, hidden_size = 128)
generator_body = ElectraModel(generator_config)
generator_head = GeneratorHead(generator_config)
# generator = Generator(generator_body, generator_head)
generator = BertForMaskedLM(config=generator_config)

discriminator_config = ElectraConfig(vocab_size = tokenizer.vocab_size)
discriminator_body = ElectraModel(discriminator_config)
discriminator_head = DiscriminatorHead(discriminator_config)
discriminator = Discriminator(discriminator_body,discriminator_head)

loss = ELECTRALoss()

In [72]:
output = generator(batch['input_ids'], batch['attention_mask'])

In [114]:
labels = batch['labels']

In [50]:
torch.argmax(output.logits, dim=-1)[0]

tensor([ 93075, 113047,  15324, 233572,  87194,   1204, 218538, 241753, 214344,
         20164,  19840,  18549, 245631,  25191, 234619, 162762, 113047,  16199,
        246091, 104670, 147656, 194834, 235421, 164938,  92980, 186719, 241159,
        187909, 160990, 131696,  52339, 248546,  77693, 109212, 246091, 191506,
        225096, 198822, 211377,  83774, 107422,  13388, 214552, 248546, 244920,
          1092, 163240, 197561, 132632, 239264,  95170, 231796, 195645, 183186,
         23979,  61435,  82915, 162762, 234560, 248588, 242596, 140390,  13429,
        121875, 209418, 243439, 121103, 194491, 160145, 207102,  97297, 151763,
        227560, 172997, 167271,  53865, 135949, 195689,  94580, 132090, 163780,
        246091,  57476,  31194, 193426, 179385,  28473, 205462,  26473, 209418,
         37666, 129153,  81636, 200171,  96147, 202614,  55027,  52096,  73989,
          9247, 193426,   5104, 172530,  20001, 105079,  13222, 114659,  78433,
          1598, 245897, 163306, 136496, 

In [31]:
tokenizer(tokenizer.special_tokens_map['mask_token'])['input_ids']

[0, 250001, 2]

In [58]:
index = 0
input = batch['input_ids'][0]
torch.argmax(output.logits, dim=-1)[0]

tensor([ 93075, 113047,  15324, 233572,  87194,   1204, 218538, 241753, 214344,
         20164,  19840,  18549, 245631,  25191, 234619, 162762, 113047,  16199,
        246091, 104670, 147656, 194834, 235421, 164938,  92980, 186719, 241159,
        187909, 160990, 131696,  52339, 248546,  77693, 109212, 246091, 191506,
        225096, 198822, 211377,  83774, 107422,  13388, 214552, 248546, 244920,
          1092, 163240, 197561, 132632, 239264,  95170, 231796, 195645, 183186,
         23979,  61435,  82915, 162762, 234560, 248588, 242596, 140390,  13429,
        121875, 209418, 243439, 121103, 194491, 160145, 207102,  97297, 151763,
        227560, 172997, 167271,  53865, 135949, 195689,  94580, 132090, 163780,
        246091,  57476,  31194, 193426, 179385,  28473, 205462,  26473, 209418,
         37666, 129153,  81636, 200171,  96147, 202614,  55027,  52096,  73989,
          9247, 193426,   5104, 172530,  20001, 105079,  13222, 114659,  78433,
          1598, 245897, 163306, 136496, 

In [117]:
predictions = torch.argmax(output.logits, dim=-1)
(input == predictions)*2-1

tensor([[-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1],
        ...,
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1, -1, -1,  ..., -1, -1, -1]])

In [126]:
torch.sum((predictions[labels != -100] == labels[labels != -100])*1.)
# generator_accuracy = torch.mean()

tensor(0.)

In [81]:
batch['labels'].view(-1).shape

torch.Size([3328])

In [76]:
output.logits.view(-1, tokenizer.vocab_size).shape

torch.Size([3328, 250002])

In [93]:
predicted_tokens = torch.argmax(output.logits, dim=-1)
discriminator_input = predicted_tokens
discriminator_predictions = discriminator(discriminator_input)
discriminator_labels = (input == predicted_tokens)*1

In [112]:
discriminator_labels[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [91]:
discriminator_predictions.shape

torch.Size([26, 128, 2])

In [102]:
torch.nn.functional.binary_cross_entropy_with_logits(discriminator_predictions, torch.nn.functional.one_hot(discriminator_labels, num_classes=2).float())

tensor(0.5442, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [111]:
discriminator_labels.shape

torch.Size([26, 128])

In [127]:
torch.argmax(discriminator_predictions, dim=-1)[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

# Training

In [59]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Create a folder
# !mkdir -p "/content/drive/My Drive/LLM-projects-models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
def training(generator, discriminator, n_epochs, train_dataloader:DataLoader, valid_dataloader:DataLoader, loss, tokenizer, lr=5e-5):

    #trained_gen = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').cuda()
    optimizer_gen = torch.optim.AdamW(
        generator.parameters(),
        lr=lr,
        eps=1e-08,
    )
    optimizer_disc = torch.optim.AdamW(
        discriminator.parameters(),
        lr=lr,
        eps=1e-08,
    )
    gen_loss = nn.CrossEntropyLoss(ignore_index = -100)
    discrim_loss = nn.BCEWithLogitsLoss()
    list_train_loss = []
    list_val_loss = []
    best_val_loss = 10**50
    for e in range(n_epochs):
        # ========== Training ==========

        # Set model to training mode
        generator.train()
        discriminator.train()

        # Tracking variables
        train_loss = 0
        for batch in tqdm(train_dataloader):
            input_ids, attention_mask, labels, _ =(
                batch["input_ids"],
                batch["attention_mask"],
                batch["labels"],
                batch["token_type_ids"]
            )
            
            optimizer_gen.zero_grad()
            optimizer_disc.zero_grad()

            generator_pred = generator(input_ids, attention_mask)
            predicted_tokens = torch.argmax(generator_pred, dim=-1)
            loss_g = gen_loss(predicted_tokens, labels)
            
            discriminator_input = predicted_tokens
            discriminator_predictions = discriminator(discriminator_input)
            discriminator_labels = (input_ids == predictions)*2-1
            loss_d = discrim_loss(discriminator_predictions, discriminator_labels)

            loss_g.backward()
            loss_d.backward()

            optimizer_gen.step()
            optimizer_disc.step()

            train_loss += loss_g.item() + loss_d.item()
        list_train_loss.append(train_loss / len(train_dataloader))
        # ========== Validation ==========
        if e%1 == 0:
            generator.eval()
            discriminator.eval()
            valid_loss = 0
            val_gen_acc = 0
            val_disc_acc = 0
            for batch in tqdm(valid_dataloader):
                batch = {k: v.cuda() for k, v in batch.items()}

                input_ids, attention_mask, labels, masked_ids =(
                    batch["input_ids"],
                    batch["attention_mask"],
                    batch["labels"],
                    batch["masked_ids"]
                )
                # Forward pass
            
                generator_pred = generator(input_ids, attention_mask)
                predicted_tokens = torch.argmax(generator_pred, dim=-1)
                loss_g = gen_loss(predicted_tokens, labels)
            
                discriminator_input = predicted_tokens
                discriminator_predictions = discriminator(discriminator_input)
                discriminator_labels = (input_ids == predictions)*2-1
                loss_d = discrim_loss(discriminator_predictions, discriminator_labels)

                valid_loss += loss_g.item() + loss_d.item()
                
                generator_accuracy = torch.mean((predicted_tokens[labels != -100] == labels[labels != -100])*1)
                discriminator_accuracy = torch.mean((discriminator_predictions == discriminator_labels)*1)
                val_gen_acc += generator_accuracy
                val_disc_acc += discriminator_accuracy
            
        list_val_loss.append(valid_loss / len(valid_dataloader))
        if list_val_loss[-1] < best_val_loss:
          best_val_loss = list_val_loss[-1]
          generator_path = f"/content/drive/My Drive/LLM-projects-models/generator_epoch{e}_lr{lr}"
          discriminator_path =  f"/content/drive/My Drive/LLM-projects-models/discriminator_epoch{e}_lr{lr}"
          torch.save(generator.state_dict(), generator_path)
          torch.save(discriminator.state_dict(), discriminator_path)


        print(
            e,
            "\n\t - Train loss: {:.4f}".format(list_train_loss[-1]),
            "\n\t - Val loss: {:.4f}".format(list_val_loss[-1]),
            "\n\t - Val gen acc: {:.4f}".format(val_gen_acc),
            "\n\t - Val disc acc: {:.4f}".format(val_disc_acc),
        )
    return list_train_loss, list_val_loss

In [61]:
list_train_loss, list_val_loss = training(generator, discriminator, 50, train_dataloader, valid_dataloader, loss, tokenizer, lr=5e-4)

  0%|          | 0/50 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([4064])) must be the same as input size (torch.Size([4064, 2]))

In [46]:
plt.figure()
x_train = [i for i in range(len(list_train_loss))]
x_val = [i for i in range(len(list_val_loss))]
plt.plot(x_train, list_train_loss)
plt.plot(x_val, list_val_loss)

NameError: name 'list_train_loss' is not defined

<Figure size 640x480 with 0 Axes>

In [ ]:
# Example of target with class indice
loss = nn.CrossEntropyLoss(reduction = "mean")
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
print(input, target,output)

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
print(input, target,output)